In [11]:
import sqlalchemy as db
from sqlalchemy import MetaData,Table
from sqlalchemy.engine import URL,reflection

In [12]:
connection_string = "DRIVER={ODBC Driver 18 for SQL Server};SERVER=localhost,1439;DATABASE=BankingDesign;UID=sa;PWD=Pass123#;encrypt=no;trust_server_certificate=yes"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
print(connection_string)

DRIVER={ODBC Driver 18 for SQL Server};SERVER=localhost,1439;DATABASE=BankingDesign;UID=sa;PWD=Pass123#;encrypt=no;trust_server_certificate=yes


In [3]:
engine = db.create_engine(connection_url)
insp = reflection.Inspector.from_engine(engine)

/tmp/ipykernel_6073/379197190.py:2: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  insp = reflection.Inspector.from_engine(engine)


# Return Indexes

In [4]:
for name in insp.get_table_names():
     for index in insp.get_indexes(name):
         print(index)

{'name': 'UQ__BankAcco__4FC8E4A0C45281C8', 'unique': True, 'column_names': ['BankAccountId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'IX_CustomerId1', 'unique': False, 'column_names': ['CustomerId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'UQ__BankTran__2E82727AB11DB584', 'unique': True, 'column_names': ['BankTransferId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'IX_ToBankAccountId', 'unique': False, 'column_names': ['ToBankAccountId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'IX_ProductId1', 'unique': False, 'column_names': ['ProductId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'UQ__Country__10D1609E8CC26505', 'unique': True, 'column_names': ['CountryId'], 'include_columns': [], 'dialect_options': {'mssql_include': []}}
{'name': 'IX_CountryId', 'unique': False, 'column_names': ['CountryId'], 'include_columns': [], '

# Return Foreign Keys

In [22]:
for name in insp.get_table_names(schema=None):
    print(name)
    for foreign_key in insp.get_foreign_keys(name):
        print(foreign_key)

BankAccount
{'name': 'FK_BankAccount_Customer_CustomerId', 'constrained_columns': ['CustomerId'], 'referred_schema': None, 'referred_table': 'Customer', 'referred_columns': ['CustomerId'], 'options': {}}
BankTransfers
{'name': 'FK_BankTransfers_BankAccount_ToBankAccountId', 'constrained_columns': ['ToBankAccountId'], 'referred_schema': None, 'referred_table': 'BankAccount', 'referred_columns': ['BankAccountId'], 'options': {}}
Book
{'name': 'FK_Book_Product_ProductId', 'constrained_columns': ['ProductId'], 'referred_schema': None, 'referred_table': 'Product', 'referred_columns': ['ProductId'], 'options': {}}
Country
Customer
{'name': 'FK_Customer_Country_CountryId', 'constrained_columns': ['CountryId'], 'referred_schema': None, 'referred_table': 'Country', 'referred_columns': ['CountryId'], 'options': {}}
Order
{'name': 'FK_Order_Customer_CustomerId', 'constrained_columns': ['CustomerId'], 'referred_schema': None, 'referred_table': 'Customer', 'referred_columns': ['CustomerId'], 'optio

# Tables

In [13]:
metadata = MetaData()
# print(metadata.tables)
# reflect db schema to MetaData
metadata.reflect(bind=engine)
print(metadata.tables)

FacadeDict({'BankAccount': Table('BankAccount', MetaData(), Column('BankAccountId', INTEGER(), table=<BankAccount>, primary_key=True, nullable=False), Column('BankAccountNumber', NVARCHAR(length=10), table=<BankAccount>, nullable=False), Column('Balance', DECIMAL(precision=19, scale=4), table=<BankAccount>, nullable=False), Column('CustomerId', INTEGER(), ForeignKey('Customer.CustomerId'), table=<BankAccount>), Column('Locked', BIT(), table=<BankAccount>, nullable=False), schema=None), 'Customer': Table('Customer', MetaData(), Column('CustomerId', INTEGER(), table=<Customer>, primary_key=True, nullable=False), Column('CustomerCode', NVARCHAR(length=5), table=<Customer>, nullable=False), Column('CompanyName', NVARCHAR(length=50), table=<Customer>, nullable=False), Column('ContactName', NVARCHAR(length=50), table=<Customer>), Column('ContactTitle', NVARCHAR(length=50), table=<Customer>), Column('Address', NVARCHAR(length=50), table=<Customer>), Column('City', NVARCHAR(length=20), table=<

# Return data for a specific table
https://docs.sqlalchemy.org/en/14/core/reflection.html

In [ ]:
engine_specific = db.create_engine(connection_url)
# metadata_specific = MetaData(schema="OnlineAuction")
metadata_specific = MetaData(schema=None)
print(metadata_specific.tables)
# reflect db schema to MetaData
# metadata_specific.reflect(bind=engine_specific)

In [72]:
# for table in metadata.tables:
#     if (table.table == "<PriceRecommendations>"):
#         print(table)
        
online_recommendation = Table("BankTransfers", metadata_specific, autoload_with=engine_specific)
[c.name for c in online_recommendation.columns]

#https://docs.sqlalchemy.org/en/14/core/constraints.html#sqlalchemy.schema.ForeignKey.target_fullname
recommendations = metadata_specific.tables["BankTransfers"]
print("BankTransfers:", recommendations)
for c in online_recommendation.columns:
    if c.foreign_keys is not None:
        for foreign_key in c.foreign_keys:
            print("FK Name:",foreign_key.name)
            print("FK Target FullName:",foreign_key.target_fullname)
        print(c.name, c.foreign_keys)

#[c for c in recommendations.columns]# [c.name for f in recommendations.foreign_keys]
print("Tables",metadata_specific.tables)
print(recommendations.foreign_keys)

BankTransfers: BankTransfers
BankTransferId set()
FromBankAccountId set()
FK Name: FK_BankTransfers_BankAccount_ToBankAccountId
FK Target FullName: BankAccount.BankAccountId
ToBankAccountId {ForeignKey('BankAccount.BankAccountId')}
Amount set()
TransferDate set()
Tables FacadeDict({'BankTransfers': Table('BankTransfers', MetaData(), Column('BankTransferId', INTEGER(), table=<BankTransfers>, primary_key=True, nullable=False), Column('FromBankAccountId', INTEGER(), table=<BankTransfers>, nullable=False), Column('ToBankAccountId', INTEGER(), ForeignKey('BankAccount.BankAccountId'), table=<BankTransfers>, nullable=False), Column('Amount', DECIMAL(precision=18, scale=2), table=<BankTransfers>, nullable=False), Column('TransferDate', DATETIME2(), table=<BankTransfers>, nullable=False), schema=None), 'BankAccount': Table('BankAccount', MetaData(), Column('BankAccountId', INTEGER(), table=<BankAccount>, primary_key=True, nullable=False), Column('BankAccountNumber', NVARCHAR(length=10), table=<

In [16]:
def return_tables(schemas,connection_url):
    tables = []
    for schema in schemas:
        engine = db.create_engine(connection_url)
        metadata = MetaData(schema=schema)
        metadata.reflect(bind=engine)
        for table in metadata.tables:
            table_name = table.split('.')[1]
            tables.append({
                "schema": schema,
                "name": table_name
            })
    return tables

# Return Table Names. Tables will be selected and then processed

In [17]:
schemas = []
schemas.append("dbo")

table_names = return_tables(schemas, connection_url)
print(table_names)




OperationalError: (pyodbc.OperationalError) ('HYT00', '[HYT00] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0) (SQLDriverConnect)')
(Background on this error at: https://sqlalche.me/e/14/e3q8)

# Generate tables

In [ ]:
def generate_table_model(table_name):
    print(table_name)

def generate_table_models(table_names):
    for table_name in table_names:
        generate_table_model(table_name)

In [ ]:
generate_table_models(table_names)